<a href="https://colab.research.google.com/github/santteegt/om-fol-timeseries/blob/master/WESAD_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WESAD - A Multimodal Dataset for Wearable Stress and Affect Detection

## Requires Dependencies

In [1]:
!pip install neurokit2 pyhrv pyarrow

In [2]:
import concurrent.futures
from datetime import timedelta

import gzip
import logging
import matplotlib as plt
import neurokit2 as nk
import numpy as np
import os
import pandas as pd
import pyhrv
import scipy.signal as scisig
import scipy.stats
import shutil
import time
from urllib.request import Request, urlopen
import zipfile

import cvxEDA

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 7]  # Bigger images

In [3]:
if not os.path.exists('cvxEDA.py'):
    !wget https://raw.githubusercontent.com/lciti/cvxEDA/master/src/cvxEDA.py

In [4]:
!ls -l

total 2224
-rw-r--r--  1 santteegt  staff       20 Jun 11 10:10 README.md
-rw-r--r--  1 santteegt  staff  1125495 Jun 11 10:33 WESAD_Data_Exploration.ipynb
drwxr-xr-x  3 santteegt  staff       96 Jun 11 10:13 __pycache__
-rw-r--r--  1 santteegt  staff     5876 Jun 11 10:12 cvxEDA.py


## Data Loader

In [6]:
class WesadDataLoader():
    """Downloads and load data from the WESAD dataset
        
        Source URI: https://uni-siegen.sciebo.de/s/pYjSgfOVs6Ntahr/download
    """
    
    LABEL = 'label'
    SIGNAL = 'signal'
    SUBJECT = 'subject'
    
    WRIST_DEV = 'wrist'
    CHEST_DEV = 'chest'
    
    DATASET_NAME = 'WESAD'
    DATASET_URI = 'https://uni-siegen.sciebo.de/s/pYjSgfOVs6Ntahr/download'
    
    def __init__(self, subject, basepath='.'):
        self.logger = logging.getLogger(WesadDataLoader.__name__)
        self.logger.info('Init...')
        self.chest_modalities = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_modalities = ['ACC', 'BVP', 'EDA', 'TEMP']
        self.mod_samp_rate = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'chest': 700}  # Hz
        WesadDataLoader.download(basepath)
        basepath = os.path.join(os.path.abspath(basepath), WesadDataLoader.DATASET_NAME, subject)
        if not os.path.isdir(basepath):
            raise Exception(f'Dataset path does not exist or is not a directory: {basepath}')
        data_file = os.path.join(basepath, f'{subject}.pkl')
        if not os.path.exists(data_file):
            raise Exception(f'Data file does not exists: {data_file}')
#         with open(subject + '.pkl', 'rb') as file:
#             data = pickle.load(file, encoding='latin1')
        self.data = pd.read_pickle(data_file)
    
    @staticmethod
    def download(basepath):
        filename = os.path.join(os.path.abspath(basepath), f'{WesadDataLoader.DATASET_NAME}.zip')
        data_folder = os.path.join(os.path.abspath(basepath), WesadDataLoader.DATASET_NAME)
        if not os.path.isdir(data_folder) and not os.path.exists(filename):
            print('Downloading dataset...')
            start = time.time()
            response = urlopen(WesadDataLoader.DATASET_URI)
            print(f'Elapsed: {time.time() - start} secs')
        if not os.path.isdir(data_folder):
            with open(filename, 'wb') as out_file:
                print('Saving dataset locally...')
                start = time.time()
                shutil.copyfileobj(response, out_file)
            out_file.close()
            print(f'Elapsed: {time.time() - start} secs')
            start = time.time()
            while not zipfile.is_zipfile(filename):
                print('Wait..')
            print('Found Zip...')
            print(f'Elapsed: {time.time() - start} secs')
            with zipfile.ZipFile(filename) as zf:
                print('Extracting files...')
                start = time.time()
                zf.extractall()
            print(f'Elapsed: {time.time() - start} secs')
            print('Done!')

    def get_labels(self):
        return self.data[WesadDataLoader.LABEL]

    def get_wrist_data(self):
        """"""
        #label = self.data[self.keys[0]]
#         assert subject == self.data[self.keys[1]]
        signal = self.data[WesadDataLoader.SIGNAL]
        wrist_data = signal[WesadDataLoader.WRIST_DEV]
        # Adding Resp modality from chest device
        wrist_data.update({'Resp': self.data[WesadDataLoader.SIGNAL][WesadDataLoader.CHEST_DEV]['Resp']})
        return wrist_data

    def get_chest_data(self):
        """"""
        signal = self.data[WesadDataLoader.SIGNAL]
        chest_data = signal[WesadDataLoader.CHEST_DEV]
        return chest_data

## Data Exploration - Initial settings

In [7]:
%%time
BASE_PATH = './'
# WesadDataLoader.download('.')
DATASET_PATH = os.path.join(BASE_PATH, WesadDataLoader.DATASET_NAME)
subjects = [dir_ for dir_ in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, dir_))]
# subjects = ['S3']
obj_data = {}

for subject in subjects:
    obj_data[subject] = WesadDataLoader(subject=subject, basepath=BASE_PATH)

CPU times: user 52.8 s, sys: 27.2 s, total: 1min 20s
Wall time: 1min 21s


In [17]:
# Checking dataset size
sampling_rate=700
window_size=60
window_shift=0.25

baseline_rec = 0
stress_rec = 0
amusement_rec = 0
total_segmented = 0
print('Subjects', obj_data.keys())
for sub in obj_data.keys():
    data = obj_data[sub].get_chest_data()
    labels = obj_data[sub].get_labels()
    baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
    stress = np.asarray([idx for idx,val in enumerate(labels) if val == 2])
    amusement = np.asarray([idx for idx,val in enumerate(labels) if val == 3])

    baseline_rec += baseline.shape[0]
    stress_rec += stress.shape[0]
    amusement_rec += amusement.shape[0]
    conditions = [baseline, stress, amusement]
    
    subtotal = 0
    for cond in conditions:
        subtotal += len(list(range(0, data['ACC'][cond].shape[0] - (sampling_rate * window_size), int(sampling_rate * window_shift))))
    print('Subject', sub, subtotal)
    total_segmented += subtotal

(baseline_rec + stress_rec + amusement_rec), total_segmented

Subjects dict_keys(['S5', 'S2', 'S3', 'S4', 'S17', 'S10', 'S11', 'S16', 'S8', 'S6', 'S7', 'S9', 'S13', 'S14', 'S15'])
Subject S5 8148
Subject S2 7764
Subject S3 7900
Subject S4 7941
Subject S17 8384
Subject S10 8388
Subject S11 8192
Subject S16 8165
Subject S8 8116
Subject S6 8088
Subject S7 8073
Subject S9 8068
Subject S13 8185
Subject S14 8189
Subject S15 8212


(23206404, 121813)

In [8]:
def get_slope(series):
    linreg = scipy.stats.linregress(np.arange(len(series)), series )
    slope = linreg[0]
    return slope


def get_freq_features(series):
    # Peak frequency is simply the frequency of maximum power
    f, Pxx = scisig.periodogram(series) # Estimate power spectral density (PSD) using a periodogram
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    # Mean freq: http://luscinia.sourceforge.net/page26/page35/page35.html
    mean_freq = np.dot(Pxx, f) / np.sum(Pxx)
    avg_power = np.sum(Pxx) / 2
    # Median freq: http://luscinia.sourceforge.net/page26/page36/page36.html
    median_freq = f[(np.cumsum(f) > avg_power).argmax()]

    return peak_freq, mean_freq, median_freq

### Compute Features

In [9]:
def compute_features(data, condition, sampling_rate=700, window_size=60, window_shift=0.25):

    index = 0
    init = time.time()

    # data cleaning
    ## ECG
    ecg_cleaned = nk.ecg_clean(data["ECG"][condition].flatten(), sampling_rate=sampling_rate)
    ## == OLD
    # ecg_rpeaks, _ = nk.ecg_peaks(ecg_cleaned, sampling_rate=sampling_rate)
    # ecg_hr = nk.signal_rate(ecg_rpeaks, sampling_rate=sampling_rate)
    ## ==
    ## EDA
    ## 5Hz lowpass filter
    eda_highcut = 5
    eda_filtered = nk.signal_filter(data['EDA'][condition].flatten(), sampling_rate=sampling_rate, highcut=eda_highcut)
    eda_cleaned = nk.standardize(eda_filtered)
    # TODO: not sure about the approach. cvxeda takes longer periods
    # phasic_tonic = nk.eda_phasic(cleaned, sampling_rate=700, method='cvxeda')
    eda_phasic_tonic = nk.eda_phasic(eda_cleaned, sampling_rate=sampling_rate)
    eda_phasic_tonic['t'] = [(1 / sampling_rate) * i for i in range(eda_phasic_tonic.shape[0])]
    eda_scr_peaks, scr_info = nk.eda_peaks(eda_phasic_tonic['EDA_Phasic'], sampling_rate=sampling_rate)
    ## EMG
    ## For 5 sec window signal
    ## More on DC Bias https://www.c-motion.com/v3dwiki/index.php/EMG:_Removing_DC_Bias
    emg_lowcut = 50
    emg_filtered_dc = nk.signal_filter(data['EMG'][condition].flatten(), sampling_rate=sampling_rate, lowcut=emg_lowcut)
    # OR 100 Hz highpass Butterworth filter followed by a constant detrending
    # filtered_dc = nk.emg_clean(chest_data_dict['EMG'][baseline].flatten(), sampling_rate=700)
    ## For 60 sec window signal
    # 50Hz lowpass filter
    emg_highcut = 50
    emg_filtered = nk.signal_filter(data['EMG'][condition].flatten(), sampling_rate=sampling_rate, highcut=emg_highcut)
    ## Resp
    ## Method biosppy important to appply bandpass filter 0.1 - 0.35 Hz
    resp_processed, _ = nk.rsp_process(data['Resp'][condition].flatten(), sampling_rate=sampling_rate, method='biosppy')

    print('Elapsed Preprocess', str(timedelta(seconds=time.time() - init)))
    init = time.time()

    chest_df_5 = pd.DataFrame() # For 5 sec window size
    chest_df = pd.DataFrame()

    window = int(sampling_rate * window_size)
    for i in range(0, data['ACC'][condition].shape[0] - window, int(sampling_rate * window_shift)):

        # ACC
        w_acc_data = data['ACC'][condition][i: window + i]
        acc_x_mean, acc_y_mean, acc_z_mean = np.mean(w_acc_data, axis=0)  # Feature
        acc_x_std, acc_y_std, acc_z_std = np.std(w_acc_data, axis=0)  # Feature
        acc_x_peak, acc_y_peak, acc_z_peak = np.amax(w_acc_data, axis=0)  # Feature
        acc_x_absint, acc_y_absint, acc_z_absint = np.abs(np.trapz(w_acc_data, axis=0))  # Feature
        xyz = np.sum(w_acc_data, axis=0)
        xyz_mean = np.mean(xyz)  # Feature
        xyz_std = np.std(xyz)  # Feature
        xyz_absint = np.abs(np.trapz(xyz))  # Feature


        # == OLD
        # ## ECG
        # w_ecg_rpeaks = ecg_rpeaks[i: window + i]
        # # HR
        # w_ecg_hr = ecg_hr[i: window + i]
        # hr_mean = np.mean(w_ecg_hr)  # Feature
        # hr_std = np.std(w_ecg_hr)  # Feature
        # # HRV Time-domain Indices
        # # HRV_MeanNN
        # # HRV_SDNN
        # # HRV_pNN50
        # # HRV_RMSSD -> Root mean square of the HRV
        # # HRV_HTI -> Triangular interpolation index
        # hrv_time = nk.hrv_time(w_ecg_rpeaks, sampling_rate=sampling_rate, show=False)
        # hrv_mean = hrv_time.loc[0, 'HRV_MeanNN']  # Feature
        # hrv_std = hrv_time.loc[0, 'HRV_SDNN']  # Feature
        # # TODO: NN50
        # # hrv_NN50 = 
        # hrv_pNN50 = hrv_time.loc[0, 'HRV_pNN50']  # Feature
        # hrv_TINN = hrv_time.loc[0, 'HRV_HTI']  # Feature
        # hrv_rms = hrv_time.loc[0, 'HRV_RMSSD']  # Feature

        # # HRV Frequency-domain Indices
        # # TODO: get NaN values within windows (*)
        # # HRV_ULF *
        # # HRV_LF *
        # # HRV_HF 
        # # HRV_VHF
        # # HRV_LFHF - Ratio LF/HF *
        # # HRV_LFn *
        # # HRV_HFn
        # hrv_freq = nk.hrv_frequency(w_ecg_rpeaks, sampling_rate=sampling_rate, ulf=(0.01, 0.04), lf=(0.04, 0.15), hf=(0.15, 0.4), vhf=(0.4, 1.))
        # hrv_ULF = hrv_freq.loc[0, 'HRV_ULF']  # Feature
        # hrv_LF = hrv_freq.loc[0, 'HRV_LF']  # Feature
        # hrv_HF = hrv_freq.loc[0, 'HRV_HF']  # Feature
        # hrv_VHF = hrv_freq.loc[0, 'HRV_VHF']  # Feature
        # hrv_lf_hf_ratio = hrv_freq.loc[0, 'HRV_LFHF']  # Feature
        # hrv_f_sum = np.nansum(np.hstack((hrv_ULF, hrv_LF, hrv_HF, hrv_VHF)))
        # # TODO: rel_f
        # # hrv_rel_f = 
        # hrv_LFn = hrv_freq.loc[0, 'HRV_LFn']  # Feature
        # hrv_HFn = hrv_freq.loc[0, 'HRV_HFn']  # Feature
        # ==

        ## ECG 
        w_ecg_cleaned = ecg_cleaned[i: window + i]
        _, ecg_info = nk.ecg_peaks(w_ecg_cleaned, sampling_rate=sampling_rate)
        w_ecg_rpeaks = ecg_info['ECG_R_Peaks']
        ecg_nni = pyhrv.tools.nn_intervals(w_ecg_rpeaks)
        # HR
        rs_hr = pyhrv.time_domain.hr_parameters(ecg_nni)
        hr_mean = rs_hr['hr_mean']  # Feature
        hr_std = rs_hr['hr_std']  # Feature
        # HRV-time
        rs_hrv = pyhrv.time_domain.nni_parameters(ecg_nni)
        hrv_mean = rs_hrv['nni_mean']  # Feature
        hrv_std = pyhrv.time_domain.sdnn(ecg_nni)['sdnn']  # Feature
        rs_nn50 = pyhrv.time_domain.nn50(ecg_nni)
        hrv_NN50 = rs_nn50['nn50']  # Feature
        hrv_pNN50 = rs_nn50['pnn50']  # Feature
        hrv_time = nk.hrv_time(w_ecg_rpeaks, sampling_rate=sampling_rate, show=False)
        hrv_TINN = hrv_time.loc[0, 'HRV_TINN']  # Feature
        hrv_rms = pyhrv.time_domain.rmssd(ecg_nni)['rmssd']  # Feature
        # HRV-freq
        hrv_freq = pyhrv.frequency_domain.welch_psd(ecg_nni, fbands={'ulf': (0.01, 0.04), 'vlf': (0.04, 0.15), 'lf': (0.15, 0.4), 'hf': (0.4, 1)}, mode='dev')
        # hrv_freq = hrv_freq.as_dict()
        hrv_freq = hrv_freq[0]
        hrv_ULF = hrv_freq['fft_abs'][0]  # Feature
        hrv_LF = hrv_freq['fft_abs'][1]  # Feature
        hrv_HF = hrv_freq['fft_abs'][2]  # Feature
        hrv_VHF = hrv_freq['fft_abs'][3]  # Feature
        hrv_lf_hf_ratio = hrv_freq['fft_ratio']  # Feature
        hrv_f_sum = hrv_freq['fft_total']  # Feature
        hrv_rel_ULF = hrv_freq['fft_rel'][0]  # Feature
        hrv_rel_LF = hrv_freq['fft_rel'][1]  # Feature
        hrv_rel_HF = hrv_freq['fft_rel'][2]  # Feature
        hrv_rel_VHF = hrv_freq['fft_rel'][3]  # Feature
        hrv_LFn = hrv_freq['fft_norm'][0]  # Feature
        hrv_HFn = hrv_freq['fft_norm'][1]  # Feature

        # EDA
        w_eda_data = eda_cleaned[i: window + i]
        w_eda_phasic_tonic = eda_phasic_tonic[i: window + i]

        eda_mean = np.mean(w_eda_data)  # Feature
        eda_std = np.std(w_eda_data)  # Feature
        eda_min = np.amin(w_eda_data)  # Feature
        eda_max = np.amax(w_eda_data)  # Feature
        # dynamic range: https://en.wikipedia.org/wiki/Dynamic_range
        eda_slope = get_slope(w_eda_data)  # Feature
        eda_drange = eda_max / eda_min  # Feature
        eda_scl_mean = np.mean(w_eda_phasic_tonic['EDA_Tonic'])  # Feature
        eda_scl_std = np.std(w_eda_phasic_tonic['EDA_Tonic'])  # Feature
        eda_scr_mean = np.mean(w_eda_phasic_tonic['EDA_Phasic'])  # Feature
        eda_scr_std = np.std(w_eda_phasic_tonic['EDA_Phasic'])  # Feature
        eda_corr_scl_t = nk.cor(w_eda_phasic_tonic['EDA_Tonic'], w_eda_phasic_tonic['t'], show=False)  # Feature
        
        eda_scr_no = eda_scr_peaks['SCR_Peaks'][i: window + i].sum()  # Feature
        # Sum amplitudes in SCR signal
        ampl = scr_info['SCR_Amplitude'][i: window + i]
        eda_ampl_sum = np.sum(ampl[~np.isnan(ampl)])  # Feature
        # TODO: 
        # eda_t_sum = 

        scr_peaks, scr_properties = scisig.find_peaks(w_eda_phasic_tonic['EDA_Phasic'], height=0)
        width_scr = scisig.peak_widths(w_eda_phasic_tonic['EDA_Phasic'], scr_peaks, rel_height=0)
        ht_scr = scr_properties['peak_heights']
        eda_scr_area = 0.5 * np.matmul(ht_scr, width_scr[1])  # Feature

        # EMG
        ## 5sec
        w_emg_data = emg_filtered_dc[i: window + i]
        emg_mean = np.mean(w_emg_data)  # Feature
        emg_std = np.std(w_emg_data)  # Feature
        emg_min = np.amin(w_emg_data)
        emg_max = np.amax(w_emg_data)
        emg_drange = emg_max / emg_min  # Feature
        emg_absint = np.abs(np.trapz(w_emg_data))  # Feature
        emg_median = np.median(w_emg_data)  # Feature
        emg_perc_10 = np.percentile(w_emg_data, 10)  # Feature
        emg_perc_90 = np.percentile(w_emg_data, 90)  # Feature
        emg_peak_freq, emg_mean_freq, emg_median_freq = get_freq_features(w_emg_data)  # Features
        # TODO: PSD -> energy in seven bands
        # emg_psd = 

        ## 60 sec
        peaks, properties = scisig.find_peaks(emg_filtered[i: window + i], height=0)
        emg_peak_no = peaks.shape[0]
        emg_peak_amp_mean = np.mean(properties['peak_heights'])  # Feature
        emg_peak_amp_std = np.std(properties['peak_heights'])  # Feature
        emg_peak_amp_sum = np.sum(properties['peak_heights'])  # Feature
        emg_peak_amp_max = np.abs(np.amax(properties['peak_heights']))
        # https://www.researchgate.net/post/How_Period_Normalization_and_Amplitude_normalization_are_performed_in_ECG_Signal
        emg_peak_amp_norm_sum = np.sum(properties['peak_heights'] / emg_peak_amp_max)  # Feature

        # Resp
        w_resp_data = resp_processed[i: window + i]
        ## Inhalation / Exhalation duration analysis
        idx = np.nan
        count = 0
        duration = dict()
        first = True
        for j in w_resp_data[~w_resp_data['RSP_Phase'].isnull()]['RSP_Phase'].to_numpy():
            if j != idx:
                if first:
                    idx = int(j)
                    duration[1] = []
                    duration [0] = []
                    first = False
                    continue
                # print('New value', j, count)
                duration[idx].append(count)
                idx = int(j)
                count = 0 
            count += 1
        resp_inhal_mean = np.mean(duration[1])  # Feature
        resp_inhal_std = np.std(duration[1])  # Feature
        resp_exhal_mean = np.mean(duration[0])  # Feature
        resp_exhal_std = np.std(duration[0])  # Feature
        resp_inhal_duration = w_resp_data['RSP_Phase'][w_resp_data['RSP_Phase'] == 1].count()
        resp_exhal_duration = w_resp_data['RSP_Phase'][w_resp_data['RSP_Phase'] == 0].count()
        resp_ie_ratio = resp_inhal_duration / resp_exhal_duration  # Feature
        resp_duration = resp_inhal_duration + resp_exhal_duration  # Feature
        resp_stretch = w_resp_data['RSP_Amplitude'].max() - w_resp_data['RSP_Amplitude'].min()  # Feature
        resp_breath_rate = len(duration[1])  # Feature
        ## Volume: area under the curve of the inspiration phase on a respiratory cycle
        resp_peaks, resp_properties = scisig.find_peaks(w_resp_data['RSP_Clean'], height=0)
        resp_width = scisig.peak_widths(w_resp_data['RSP_Clean'], resp_peaks, rel_height=0)
        resp_ht = resp_properties['peak_heights']        
        resp_volume = 0.5 * np.matmul(resp_ht, resp_width[1])  # Feature

        # Temp
        w_temp_data = data['Temp'][condition][i: window + i].flatten()
        temp_mean = np.mean(w_temp_data)  # Feature
        temp_std = np.std(w_temp_data)  # Feature
        temp_min = np.amin(w_temp_data)  # Feature
        temp_max = np.amax(w_temp_data)  # Feature
        temp_drange = temp_max / temp_min  # Feature
        temp_slope = get_slope(w_temp_data.ravel())  # Feature


        # chest_df_5 = chest_df_5.append({
        #     'ACC_x_mean': acc_x_mean, 'ACC_y_mean': acc_y_mean, 'ACC_z_mean': acc_z_mean, 'ACC_xzy_mean': xyz_mean,
        #     'ACC_x_std': acc_x_std, 'ACC_y_std': acc_y_std, 'ACC_z_std': acc_z_std, 'ACC_xyz_std': xyz_std,
        #     'ACC_x_absint': acc_x_absint, 'ACC_y_absint': acc_y_absint, 'ACC_z_absint': acc_z_absint, 'ACC_xyz_absint': xyz_absint,
        #     'ACC_x_peak': acc_x_peak, 'ACC_y_peak': acc_y_peak, 'ACC_z_peak': acc_z_peak,
        #     'EMG_mean': emg_mean, 'EMG_std': emg_std, 'EMG_drange': emg_drange, 'EMG_absint': emg_absint, 'EMG_median': emg_median, 'EMG_perc_10': emg_perc_10,
        #     'EMG_perc_90': emg_perc_90, 'EMG_peak_freq': emg_peak_freq, 'EMG_mean_freq': emg_mean_freq, 'EMG_median_freq': emg_median_freq
        # }, ignore_index=True)

        chest_df = chest_df.append({
            'ACC_x_mean': acc_x_mean, 'ACC_y_mean': acc_y_mean, 'ACC_z_mean': acc_z_mean, 'ACC_xzy_mean': xyz_mean,
            'ACC_x_std': acc_x_std, 'ACC_y_std': acc_y_std, 'ACC_z_std': acc_z_std, 'ACC_xyz_std': xyz_std,
            'ACC_x_absint': acc_x_absint, 'ACC_y_absint': acc_y_absint, 'ACC_z_absint': acc_z_absint, 'ACC_xyz_absint': xyz_absint,
            'ACC_x_peak': acc_x_peak, 'ACC_y_peak': acc_y_peak, 'ACC_z_peak': acc_z_peak,
            'ECG_hr_mean': hr_mean, 'ECG_hr_std': hr_std, 'ECG_hrv_NN50': hrv_NN50, 'ECG_hrv_pNN50': hrv_pNN50, 'ECG_hrv_TINN': hrv_TINN, 'ECG_hrv_RMS': hrv_rms,
            'ECG_hrv_ULF': hrv_ULF, 'ECG_hrv_LF': hrv_LF, 'ECG_hrv_HF': hrv_HF, 'ECG_hrv_VHF': hrv_VHF, 'ECG_hrv_LFHF_ratio': hrv_lf_hf_ratio, 'ECG_hrv_f_sum': hrv_f_sum,
            'ECG_hrv_rel_ULF': hrv_rel_ULF, 'ECG_hrv_rel_LF': hrv_rel_LF, 'ECG_hrv_rel_HF': hrv_rel_HF, 'ECG_hrv_rel_VHF': hrv_rel_VHF, 'ECG_hrv_LFn': hrv_LFn, 'ECG_hrv_HFn': hrv_HFn,
            'EDA_mean': eda_mean, 'EDA_std': eda_std, 'EDA_mean': eda_mean, 'EDA_min': eda_min, 'EDA_max': eda_max, 'EDA_slope': eda_slope,
            'EDA_drange': eda_drange, 'EDA_SCL_mean': eda_scl_mean, 'EDA_SCL_std': eda_scl_mean, 'EDA_SCR_mean': eda_scr_mean, 'EDA_SCR_std': eda_scr_std,
            'EDA_corr_SCL_t': eda_corr_scl_t, 'EDA_SCR_no': eda_scr_no, 'EDA_ampl_sum': eda_ampl_sum, 'EDA_scr_area': eda_scr_area,
            'EMG_mean': emg_mean, 'EMG_std': emg_std, 'EMG_drange': emg_drange, 'EMG_absint': emg_absint, 'EMG_median': emg_median, 'EMG_perc_10': emg_perc_10,
            'EMG_perc_90': emg_perc_90, 'EMG_peak_freq': emg_peak_freq, 'EMG_mean_freq': emg_mean_freq, 'EMG_median_freq': emg_median_freq,
            'EMG_peak_no': emg_peak_no, 'EMG_peak_amp_mean':  emg_peak_amp_mean, 'EMG_peak_amp_std':  emg_peak_amp_std, 'EMG_peak_amp_sum':  emg_peak_amp_sum,
            'EMG_peak_amp_norm_sum':  emg_peak_amp_norm_sum,
            'RESP_inhal_mean': resp_inhal_mean, 'RESP_inhal_std': resp_inhal_std, 'RESP_exhal_mean': resp_exhal_mean, 'RESP_exhal_std': resp_exhal_std,
            'RESP_ie_ratio': resp_ie_ratio, 'RESP_duration': resp_duration, 'RESP_stretch': resp_stretch, 'RESP_breath_rate': resp_breath_rate, 'RESP_volume': resp_volume,
            'TEMP_mean': temp_mean, 'TEMP_std': temp_std, 'TEMP_min': temp_min, 'TEMP_max': temp_max, 'TEMP_drange': temp_drange, 'TEMP_slope': temp_slope
        }, ignore_index=True)


        # index += 1
        # if index % 10 == 0:
        #     break
    
    print('Elapsed Process', condition.shape[0], str(timedelta(seconds=time.time() - init)))
    return chest_df, chest_df_5


## Chest-worn device - Dataset Generation

In [10]:
def process_subject(subject_data, cond_to_process, max_workers=6):
    rs = dict()

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_label = {executor.submit(compute_features, subject_data, cond): label for label, cond in cond_to_process}
        for future in concurrent.futures.as_completed(future_to_label):
            label = future_to_label[future]
            try:
                data, _ = future.result()
                print(label, data.shape)
                rs[label] = data
            except Exception as exc:
                print('%r generated an exception: %s' % (label, exc))
    return rs

In [15]:
subjects = ['S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']
for subject in subjects:
    print('Subject', subject)
    chest_data_dict = obj_data[subject].get_chest_data()
    labels = obj_data[subject].get_labels()
    chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
    print(chest_dict_length)

    # Get labels
    baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
    stress = np.asarray([idx for idx,val in enumerate(labels) if val == 2])
    amusement = np.asarray([idx for idx,val in enumerate(labels) if val == 3])

    print("Baseline:", chest_data_dict['ECG'][baseline].shape)
    print("Stress:", chest_data_dict['ECG'][stress].shape)
    print("Amusement:", chest_data_dict['ECG'][amusement].shape)

    # Process Subject
    to_process = zip(['baseline', 'stress', 'amusement'], [baseline, stress, amusement])
    # to_process = zip(['baseline'], [baseline])
    %time subject_data = process_subject(chest_data_dict, cond_to_process=to_process)

    ## Labeling
    subject_data['baseline']['label'] = 1
    subject_data['baseline']['subject'] = subject
    subject_data['stress']['label'] = 2
    subject_data['stress']['subject'] = subject
    subject_data['amusement']['label'] = 3
    subject_data['amusement']['subject'] = subject
    ## Storing
    dfs = [v for k, v in subject_data.items()]
    df_subject = pd.concat(dfs)
    print('Generated dataset for', subject, df_subject.shape)
    df_subject.head()
    df_subject.reset_index().to_feather(f'{subject}.feather')

Subject S7
{'ACC': 3666600, 'ECG': 3666600, 'EMG': 3666600, 'EDA': 3666600, 'Temp': 3666600, 'Resp': 3666600}
Baseline: (830200, 1)
Stress: (448000, 1)
Amusement: (260401, 1)
Elapsed Preprocess 0:00:03.382145
Elapsed Preprocess 0:00:04.232839
Elapsed Preprocess 0:00:10.452032
Elapsed Process 260401 0:02:40.979362
amusement (1249, 77)
Elapsed Process 448000 0:04:29.071290
stress (2320, 77)
Elapsed Process 830200 0:07:14.459127
baseline (4504, 77)
CPU times: user 38min 59s, sys: 7min 4s, total: 46min 4s
Wall time: 7min 24s
Generated dataset for S7 (8073, 79)
Subject S8
{'ACC': 3826200, 'ECG': 3826200, 'EMG': 3826200, 'EDA': 3826200, 'Temp': 3826200, 'Resp': 3826200}
Baseline: (818300, 1)
Stress: (469000, 1)
Amusement: (258999, 1)
Elapsed Preprocess 0:00:02.288148
Elapsed Preprocess 0:00:02.823671
Elapsed Preprocess 0:00:04.078425
Elapsed Process 258999 0:02:34.723430
amusement (1240, 77)
Elapsed Process 469000 0:04:38.775093
stress (2440, 77)
Elapsed Process 818300 0:07:15.700035
baselin

In [16]:
# Download files If running in Google Colab
# from google.colab import files
# [files.download(file) for file in os.listdir('.') if file.endswith('feather')]

## Chest-worn device - Data Analysis

In [ ]:
subject ='S3'
chest_data_dict = obj_data[subject].get_chest_data()
labels = obj_data[subject].get_labels()
chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
print(chest_dict_length)

In [ ]:
%%time
# Get labels
# labels = obj_data[subject].get_labels()
baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
stress = np.asarray([idx for idx,val in enumerate(labels) if val == 2])
amusement = np.asarray([idx for idx,val in enumerate(labels) if val == 3])

print("Baseline:", chest_data_dict['ECG'][baseline].shape)
print("Stress:", chest_data_dict['ECG'][stress].shape)
print("Amusement:", chest_data_dict['ECG'][amusement].shape)

In [ ]:
# For dev purposes
# df_data, _ = compute_features(chest_data_dict, baseline)
# df_data.head()

In [ ]:
# For dev purposes
# %%time
# to_process = zip(['baseline', 'stress', 'amusement'], [baseline, stress, amusement])
# # to_process = zip(['baseline'], [baseline])

# rs = dict()

# with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
#     future_to_label = {executor.submit(compute_features, chest_data_dict, cond): label for label, cond in to_process}
#     for future in concurrent.futures.as_completed(future_to_label):
#         label = future_to_label[future]
#         try:
#             data, _ = future.result()
#             print(label, data.shape)
#             rs[label] = data
#         except Exception as exc:
#             print('%r generated an exception: %s' % (label, exc))

# rs['baseline']['label'] = 1
# rs['stress']['label'] = 2
# rs['amusement']['label'] = 3
# dfs = [v for k, v in rs.items()]

# df_s3 = pd.concat(dfs)
# df_s3.describe()

### Loading data with Neurokit

In [ ]:
df, bio_info = nk.bio_process(ecg=chest_data_dict["ECG"][baseline].flatten(),
                              rsp=chest_data_dict['Resp'][baseline].flatten(),
                              emg=chest_data_dict["EMG"][baseline].flatten(),
                              eda=chest_data_dict["EDA"][baseline].flatten(),
                              sampling_rate=700)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

### Feature Engineering

In [ ]:
# ECG

# HR
cleaned = nk.ecg_clean(chest_data_dict["ECG"][baseline].flatten(), sampling_rate=700)
rpeaks, info = nk.ecg_peaks(cleaned, sampling_rate=700, correct_artifacts=True)
# ecg_heart_rate = nk.signal_rate(rpeaks, sampling_rate=700)
# ecg_heart_rate
# Features Per Window

# HRV Time-domain Indices
# HRV_MeanNN
# HRV_SDNN
# HRV_pNN50
# HRV_RMSSD -> Root mean square of the HRV
# HRV_TINN -> Triangular interpolation index
# HRV_HTI -> Triangular interpolation index


hrv_time = nk.hrv_time(rpeaks[0:42000], sampling_rate=700, show=False)
hrv_time


# hrv_time.loc[0, 'HRV_MeanNN'], hrv_time.loc[0, 'HRV_SDNN'], hrv_time.loc[0, 'HRV_pNN50'], hrv_time.loc[0, 'HRV_RMSSD'], hrv_time.loc[0, 'HRV_HTI']

# TODO: get NaN values within windows
# HRV Frequency-domain Indices
# HRV_ULF *
# HRV_LF *
# HRV_HF 
# HRV_VHF
# HRV_LFHF - Ratio LF/HF *
# HRV_LFn *
# HRV_HFn

# hrv_freq = nk.hrv_frequency(rpeaks[0:84000], sampling_rate=700, ulf=(0.01, 0.04), lf=(0.04, 0.15), hf=(0.15, 0.4), vhf=(0.4, 1), show=True)
# hrv_freq = nk.hrv_frequency(rpeaks[0:42000], sampling_rate=700)
# hrv_freq

In [ ]:
!pip install biosppy pyhrv

In [ ]:
import pyhrv

In [ ]:
# %%time
cleaned = nk.ecg_clean(chest_data_dict["ECG"][baseline].flatten()[0:42000], sampling_rate=700)
rpeaks, info = nk.ecg_peaks(cleaned, sampling_rate=700)

nn_int = pyhrv.tools.nn_intervals(info['ECG_R_Peaks'])

# HR
rs = pyhrv.time_domain.hr_parameters(nn_int)
print(rs['hr_mean'], rs['hr_std'])

#HRV
rs_hrv = pyhrv.time_domain.nni_parameters(nn_int)
print(rs_hrv['nni_mean'], pyhrv.time_domain.sdnn(nn_int)['sdnn'])
print(pyhrv.time_domain.nn50(nn_int)) # nn50, pnn50
hrv_time = nk.hrv_time(rpeaks, sampling_rate=700, show=False)
print(hrv_time.loc[0, ['HRV_TINN']])
print(pyhrv.time_domain.rmssd(nn_int)['rmssd'])

rs = pyhrv.frequency_domain.welch_psd(nn_int, fbands={'ulf': (0.01, 0.04), 'vlf': (0.04, 0.15), 'lf': (0.15, 0.4), 'hf': (0.4, 1)}, mode='dev')
rs = rs[0]
rs['fft_abs'], rs['fft_ratio'], rs['fft_total'], rs['fft_rel'], rs['fft_norm']

In [ ]:
# from biosppy.signals.ecg import ecg
# ecg(chest_data_dict["ECG"][baseline].flatten())[1:3]

import  pyhrv.hrv

# fbands = {'ulf': (0.0, 0.1), 'vlf': (0.1, 0.2), 'lf': (0.2, 0.3), 'hf': (0.3, 0.4)}
%time results = pyhrv.hrv(signal=chest_data_dict["ECG"][baseline].flatten()[0:42000], sampling_rate=700, plot_ecg=False, plot_tachogram=False, show=False, \
                          fbands={'ulf': (0.01, 0.04), 'vlf': (0.04, 0.15), 'lf': (0.15, 0.4), 'hf': (0.4, 1)}, kwargs_ecg_plot={'mode': 'dev'}, kwargs_tachogram={'show': False})

# results = hrv(signal=signal)

# Print all the parameters keys and values individually
for key in results.keys():
   print(key, results[key])

In [ ]:
# Resp
# Method biosppy important to appply bandpass filter 0.1 - 0.35 Hz
# cleaned = nk.rsp_clean(chest_data_dict['Resp'][stress].flatten(), sampling_rate=700, method='biosppy')
# rsp_peaks, info = nk.rsp_peaks(cleaned, sampling_rate=700)

# Method biosppy important to appply bandpass filter 0.1 - 0.35 Hz
rsp_signals, info = nk.rsp_process(chest_data_dict['Resp'][stress].flatten(), sampling_rate=700, method='biosppy')
# fig = nk.rsp_plot(rsp_signals[0:42000]) 

i_duration = rsp_signals[0:42000]['RSP_Phase'][rsp_signals['RSP_Phase'] == 1].count()
e_duration = rsp_signals[0:42000]['RSP_Phase'][rsp_signals['RSP_Phase'] == 0].count()
print('I|E|I/E|RespDuration', i_duration, e_duration, i_duration / e_duration, (i_duration + e_duration))


idx = np.nan
count = 0
duration = dict()
first = True
for i in rsp_signals[0:42000][~rsp_signals[0:42000]['RSP_Phase'].isnull()]['RSP_Phase'].to_numpy():
    if i != idx:
        if first:
            idx = int(i)
            duration[1] = []
            duration [0] = []
            first = False
            continue
        # print('New value', i, count)
        duration[idx].append(count)
        idx = int(i)
        count = 0 
    count += 1

print(duration)
print("Inhalation", np.sum(duration[1]), np.mean(duration[1]), np.std(duration[1]), "Exhalation", np.sum(duration[0]), np.mean(duration[0]), np.std(duration[0]))
print("Breath rate", len(duration[1]))
print("Stretch", rsp_signals[0:42000]['RSP_Amplitude'].max() - rsp_signals[0:42000]['RSP_Amplitude'].min())



resp_peaks, resp_properties = scisig.find_peaks(rsp_signals[0:42000]['RSP_Clean'], height=0)
resp_width = scisig.peak_widths(rsp_signals[0:42000]['RSP_Clean'], resp_peaks, rel_height=0)
resp_ht = resp_properties['peak_heights']
volume = 0.5 * np.matmul(resp_ht, resp_width[1])  # Feature

print("Volume", volume)

# rsp_signals


In [ ]:
E# EDA
filtered_5hz = nk.signal_filter(chest_data_dict['EDA'][baseline].flatten(), sampling_rate=700, highcut=5)
# cleaned = nk.eda_clean(chest_data_dict['EDA'][baseline].flatten(), sampling_rate=700)
cleaned = nk.standardize(filtered_5hz)
# phasic_tonic = nk.eda_phasic(nk.standardize(cleaned), sampling_rate=700, method='cvxeda')
# TODO: not sure about the approach. cvxeda takes longer periods
# phasic_tonic = nk.eda_phasic(cleaned, sampling_rate=700, method='cvxeda')
phasic_tonic = nk.eda_phasic(cleaned, sampling_rate=700)

# phasic_tonic = nk.eda_phasic(cleaned, sampling_rate=700)
# phasic_tonic

# Features Per Window

phasic_tonic['t'] = [(1 / 700) * i for i in range(phasic_tonic.shape[0])]

# Correlation
nk.cor(phasic_tonic['EDA_Tonic'][420000:420000 + 42000], phasic_tonic['t'][0:42000], show=False)

# SCR
scr_peaks, info = nk.eda_peaks(phasic_tonic['EDA_Phasic'], sampling_rate=700)
scr_peaks['SCR_Peaks'][:10]
# scr_peaks['SCR_Peaks'][0:42000].sum()
# # Sum Amplitudes in SCR signal
# ampl = info['SCR_Amplitude'][0:42000]
# np.sum(ampl[~np.isnan(ampl)])

# 'SCR_RiseTime' 'SCR_RecoveryTime'
# info

In [ ]:
info['SCR_Peaks'][:10]

In [ ]:
from scipy.signal import find_peaks, peak_widths

# peaks,properties = find_peaks(cleaned, height=0)
# # peaks,properties = find_peaks(chest_data_dict['EDA'][baseline].flatten())
# peaks[:20]
# # properties

peaks, properties = find_peaks(cleaned[0:42000], height=0)
width_scr = peak_widths(cleaned, peaks, rel_height=0)
ht_scr = properties['peak_heights']
ar_scr = 0.5*np.matmul(ht_scr,width_scr[1])
ar_scr

In [ ]:
# nk.signal_psd??
# scisig.periodogram?
np.ravel?

In [ ]:
# EMG
# More on DC Bias https://www.c-motion.com/v3dwiki/index.php/EMG:_Removing_DC_Bias
filtered_dc = nk.signal_filter(chest_data_dict['EMG'][baseline].flatten(), sampling_rate=700, lowcut=50)
# OR 100 Hz highpass Butterworth filter followed by a constant detrending
# filtered_dc = nk.emg_clean(chest_data_dict['EMG'][baseline].flatten(), sampling_rate=700)



# 5 sec window

# Peak frequency is simply the frequency of maximum power
f, Pxx = scisig.periodogram(filtered_dc[0:3500])
psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
peak_freq = psd_dict[max(psd_dict.keys())]
# Mean freq: http://luscinia.sourceforge.net/page26/page35/page35.html
mean_freq = np.dot(Pxx, f) / np.sum(Pxx)
avg_power = np.sum(Pxx) / 2
# Median freq: http://luscinia.sourceforge.net/page26/page36/page36.html
median_freq = f[(np.cumsum(f) > avg_power).argmax()]

# peak_freq

# TODO: verify that we're using the correct method
# rs['Power']
# rs = nk.signal_psd(filtered_dc[0:3500], sampling_rate=700, method='welch', min_frequency=0, max_frequency=350, show=False)
# rs
# rs = nk.signal_power(filtered_dc[0:3500], list(range(0, 350, 50)), sampling_rate=700, continuous=True, show=False, min_frequency=0, max_frequency=350, method='welch')
# rs

# 60 sec window
## lowpass filter
# filtered_50hz = nk.signal_filter(chest_data_dict['EMG'][baseline].flatten(), sampling_rate=700, highcut=50)
# # emg, info = nk.emg_process(filtered_50hz, sampling_rate=700)

# peaks, properties = find_peaks(filtered_50hz[0:42000], height = 0)
# mn_pk_amp = np.mean(properties['peak_heights'])
# std_pk_amp = np.std(properties['peak_heights'])
# sum_pk_amp = np.sum(properties['peak_heights'])
# max_pk_amp = np.abs(np.amax(properties['peak_heights']))
# norm_sum_amp = np.sum(properties['peak_heights'] / max_pk_amp)

# mn_pk_amp, std_pk_amp, sum_pk_amp, peaks.shape[0], norm_sum_amp
# peaks.shape


# emg_amplitude = nk.emg_amplitude(filtered_dc)
# emg_amplitude



# TODO: peaks?
# emg['EMG_Activity'][0:42000].sum()

### Cardiac Activity (ECG)

In [ ]:
# signals, info = nk.ecg_process(df['ECG_Raw'], sampling_rate=700)
signals, info = nk.ecg_process(chest_data_dict['ECG'][baseline].flatten(), sampling_rate=700)
_ = nk.ecg_plot(signals)

signals, info = nk.ecg_process(chest_data_dict['ECG'][stress].flatten(), sampling_rate=700)
_ = nk.ecg_plot(signals)

signals, info = nk.ecg_process(chest_data_dict['ECG'][amusement].flatten(), sampling_rate=700)
_ = nk.ecg_plot(signals)

### Respiration (Resp)

In [ ]:
# signals, info = nk.rsp_process(df['RSP_Raw'], sampling_rate=700)
signals, info = nk.rsp_process(chest_data_dict['Resp'][baseline].flatten(), sampling_rate=700)
_ = nk.rsp_plot(signals)
signals, info = nk.rsp_process(chest_data_dict['Resp'][stress].flatten(), sampling_rate=700)
_ = nk.rsp_plot(signals)
signals, info = nk.rsp_process(chest_data_dict['Resp'][amusement].flatten(), sampling_rate=700)
_ = nk.rsp_plot(signals)

### Electromyography (EMG)

*evaluating and recording the electrical activity produced by skeletal muscles*

In [ ]:
signals, info = nk.emg_process(chest_data_dict['EMG'][baseline].flatten(), sampling_rate=700)
_ = nk.emg_plot(signals)
signals, info = nk.emg_process(chest_data_dict['EMG'][stress].flatten(), sampling_rate=700)
_ = nk.emg_plot(signals)
signals, info = nk.emg_process(chest_data_dict['EMG'][amusement].flatten(), sampling_rate=700)
_ = nk.emg_plot(signals)

### Electrodermal Activity (EDA)

*Measure of neurally mediated effects on sweat gland permeability, observed as changes in the resistance of the skin to a small electrical current, or as differences in the electrical potential between different parts of the skin.*

In [ ]:
signals, info = nk.eda_process(chest_data_dict['EDA'][baseline].flatten(), sampling_rate=700)
_ = nk.eda_plot(signals)
signals, info = nk.eda_process(chest_data_dict['EDA'][stress].flatten(), sampling_rate=700)
_ = nk.eda_plot(signals)
signals, info = nk.eda_process(chest_data_dict['EDA'][amusement].flatten(), sampling_rate=700)
_ = nk.eda_plot(signals)

## Wrist-band device - Data Analysis

In [ ]:
wrist_data_dict = obj_data[subject].get_wrist_data()
wrist_dict_length = {key: len(value) for key, value in wrist_data_dict.items()}
print(wrist_dict_length)

### Commented code

In [ ]:
# def get_labels_by_sampling_rate(labels, condition_types, sampling_rate):
#     rate = int(np.ceil(700 / sampling_rate))

#     j = 0
#     new_labels = []
#     each700 = 0
#     count = 0
#     for j in range(len(labels)):
#         each700 += 1
#         if ((j % rate) == 0):
#             new_labels.append(labels[j])
#             count += 1
#         if ((each700) == 700):
#             if (count < sampling_rate):
#                 if ((j % rate) == 0):
#                     new_labels.append(labels[j+1])
#                 else:
#                     new_labels.append(labels[j])
#             count = 0
#             each700 = 0

#     # indexes = np.ndarray((0,))
#     # for cond in condition_types:
#     #     indexes = np.concatenate((indexes, np.asarray([idx for idx,val in enumerate(labels) if val == cond])), axis=0)
#     # return indexes
#     baseline = np.asarray([idx for idx,val in enumerate(new_labels) if val == 1])
#     stress = np.asarray([idx for idx,val in enumerate(new_labels) if val == 2])
#     amusement = np.asarray([idx for idx,val in enumerate(new_labels) if val == 3])
#     return (baseline, stress, amusement)

In [ ]:
# baseline_64, stress_64, amusement_64 = get_labels_by_sampling_rate(labels, [1, 2, 3], 64)

In [ ]:
# baseline_64.shape, stress_64.shape, amusement_64.shape

In [ ]:
# labels_64 = np.hstack((baseline_64, stress_64, amusement_64))
# assert(labels_64.shape[0] == (baseline_64.shape[0] + stress_64.shape[0] + amusement_64.shape[0]))

In [ ]:
# baseline_32, stress_32, amusement_32 = get_labels_by_sampling_rate(labels, [1, 2, 3], 32)
# labels_32 = np.hstack((baseline_32, stress_32, amusement_32))
# baseline_4, stress_4, amusement_4 = get_labels_by_sampling_rate(labels, [1, 2, 3], 4)
# labels_4 = np.hstack((baseline_4, stress_4, amusement_4))

In [ ]:
# Get labels
# baseline_64 = np.asarray([idx for idx,val in enumerate(wrist_labels_64) if val == 1])
# stress_64 = np.asarray([idx for idx,val in enumerate(wrist_labels_64) if val == 2])
# amusement_64 = np.asarray([idx for idx,val in enumerate(wrist_labels_64) if val == 3])

# print("Baseline:", wrist_data_dict['BVP'][baseline_64].shape)
# print("Stress:", wrist_data_dict['BVP'][stress_64].shape)
# print("Amusement:", wrist_data_dict['BVP'][amusement_64].shape)

### Loading Data utils

In [ ]:
class EDAUtils():

    mod_samp_rate = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'Resp': 700, 'label': 700}

    WINDOW_SIZE = 60 # in seconds

    @staticmethod
    def filter_signal_fir(data, sampling_rate, cutoff=0.4, numtaps=64):
        """
            Applies a low-pass filter to accelerometer data
            # https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py

        """
        # f = cutoff / (sampling_rate / 2.0)
        nyq = 0.5 * sampling_rate
        normal_cutoff = cutoff / nyq
        FIR_coeff = scisig.firwin(numtaps, normal_cutoff)

        return scisig.lfilter(FIR_coeff, 1, data)

    @staticmethod
    def butter_lowpass_filter(data, sampling_rate, cutoff, order=5):
        """
            Gets filtered data using a low-pass butterworth filter (cutoff: (Hz), fs: (Hz), order: (?))
            Finally, applies a low-pass butterworth filter to EDA data
            # https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/load_files.py
        """

        nyq = 0.5 * sampling_rate # Nyquist Frequecy (half-cycles / sample) for digital filters
        normal_cutoff = cutoff / nyq
        b, a = scisig.butter(order, normal_cutoff, btype='lowpass', analog=False)

        # Filtering Helper functions
        y = scisig.lfilter(b, a, data)
        return y

    @staticmethod
    def eda_stats(data, sampling_rate):
        """
            Uses the CVXEDA Convex optimization approach to EDA signal processing
            # https://github.com/lciti/cvxEDA/blob/master/src/cvxEDA.py
        """
        zscore = (data - data.mean()) / data.std()
        [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(zscore, 1. / sampling_rate)
        return [r, p, t, l, d, e, obj]

    @staticmethod
    def get_net_acc(data):
        """
            Get Euclidean norm of ACC modality across the three axes
        """
        return data.apply(lambda data: np.sqrt(data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2), axis=1)

    @staticmethod
    def get_stats(data, label=-1):
        mean_features = np.mean(data)
        std_features = np.std(data)
        min_features = np.amin(data)
        max_features = np.amax(data)

        features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                    'label': label}
        return features

    @staticmethod
    def get_peak_freq(x):
        """
            TODO:
        """
        f, Pxx = scisig.periodogram(x, fs=8)
        psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
        peak_freq = psd_dict[max(psd_dict.keys())]
        return peak_freq

    @staticmethod
    def get_slope(series):
        linreg = scipy.stats.linregress(np.arange(len(series)), series )
        slope = linreg[0]
        return slope

    def downsample(data, sampling_rate, label):
        """
            As labels were set using a sampling freq of 700 Hz, it is considered as reference frequency due to
            encompass the lesser ones in its resolution.
        """
        # no_windows = int(data.shape[0] / (sampling_rate * EDAUtils.WINDOW_SIZE))
        samples = []
        
        window_size = int(sampling_rate * EDAUtils.WINDOW_SIZE)
        window_shift = 0.25

        for i in range(0, data.shape[0] - window_size, sampling_rate * window_shift):
            # Get window of data
            # w = data[window_size * i: window_size * (i + 1)].copy()
            w = data[i: window_size + i].copy()

            # Add/Calc acc norm
            w['ACC_net'] = EDAUtils.get_net_acc(w) # Feature
            
            # Calculate stats for window
            wstats = EDAUtils.get_stats(w, label=label) # Feature

            # Seperating sample and label
            x = pd.DataFrame(wstats).drop('label', axis=0)
            y = x['label'][0]
            x.drop('label', axis=1, inplace=True)

            feat_names = []
            for row in x.index:
                for col in x.columns:
                    feat_names.append('_'.join([row, col]))

            wdf = pd.DataFrame(x.values.flatten()).T
            wdf.columns = feat_names
            wdf = pd.concat([wdf, pd.DataFrame({'label': y}, index=[0])], axis=1)
            
            # More feats
            wdf['BVP_peak_freq'] = EDAUtils.get_peak_freq(w['BVP'].dropna()) # Feature
            # TODO: Empty cases 
            # print('TEMP', w['TEMP'].dropna())
            wdf['TEMP_slope'] = EDAUtils.get_slope(w['TEMP'].dropna()) # Feature
            samples.append(wdf)

        return pd.concat(samples)


### Compute Features

In [ ]:
eda_df = pd.DataFrame(wrist_data_dict['EDA'], columns=['EDA'])
bvp_df = pd.DataFrame(wrist_data_dict['BVP'], columns=['BVP'])
acc_df = pd.DataFrame(wrist_data_dict['ACC'], columns=['ACC_x', 'ACC_y', 'ACC_z'])
temp_df = pd.DataFrame(wrist_data_dict['TEMP'], columns=['TEMP'])
resp_df = pd.DataFrame(wrist_data_dict['Resp'], columns=['Resp'])

label_df = pd.DataFrame(labels, columns=['label'])

#### Signal pre-processing: Applying filters

In [ ]:
# Signal preprocessing - Applying filters
eda_df['EDA'] = EDAUtils.butter_lowpass_filter(eda_df['EDA'], EDAUtils.mod_samp_rate['EDA'], 1., 6)

for _ in acc_df.columns:
    acc_df[_] = EDAUtils.filter_signal_fir(acc_df.values, EDAUtils.mod_samp_rate['ACC'])

#### Index to datetime based on sampling freq

In [ ]:
eda_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['EDA']) * i for i in range(eda_df.shape[0])], unit='s')
bvp_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['BVP']) * i for i in range(bvp_df.shape[0])], unit='s')
acc_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['ACC']) * i for i in range(acc_df.shape[0])], unit='s')
temp_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['TEMP']) * i for i in range(temp_df.shape[0])], unit='s')
resp_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['Resp']) * i for i in range(resp_df.shape[0])], unit='s')

label_df.index = pd.to_datetime([(1 / EDAUtils.mod_samp_rate['label']) * i for i in range(label_df.shape[0])], unit='s')

#### Getting extra features from EDA Signal

In [ ]:
## SMNA -> Sudomotor Nerve Activity -> https://www.centropiaggio.unipi.it/sites/default/files/greco2015cvxeda.pdf
scr_phasic, smna, scl_tonic, _, _, _, _ = EDAUtils.eda_stats(eda_df['EDA'], EDAUtils.mod_samp_rate['EDA'])
eda_df['EDA_scr'] = scr_phasic # Feature
eda_df['EDA_smna'] = smna
eda_df['EDA_scl'] = scl_tonic # Feature

#### Joining modalities

In [ ]:
df_wrist = eda_df.join(bvp_df, how='outer')
df_wrist = df_wrist.join(temp_df, how='outer')
df_wrist = df_wrist.join(acc_df, how='outer')
df_wrist = df_wrist.join(resp_df, how='outer')
df_wrist = df_wrist.join(label_df, how='outer')

In [ ]:
eda_df.shape, bvp_df.shape, acc_df.shape, temp_df.shape, resp_df.shape, label_df.shape

In [ ]:
df_wrist.shape

In [ ]:
df_wrist['label'].isnull().sum()

In [ ]:
df_wrist.shape[0] - df_wrist['label'].isnull().sum()

In [ ]:
df_wrist.head()

#### Filling NA Values on labels

In [ ]:
df_wrist['label'] = df_wrist['label'].fillna(method='bfill')
df_wrist.reset_index(drop=True, inplace=True)

In [ ]:
df_wrist.head()

In [ ]:
df_wrist['label'].isnull().sum()

#### Split Dataset based on Affective Conditions

In [ ]:
df_wrist_grouped = df_wrist.groupby('label')
baseline_wrist = df_wrist_grouped.get_group(1)
stress_wrist = df_wrist_grouped.get_group(2)
amusement_wrist = df_wrist_grouped.get_group(3)
baseline_wrist.shape, stress_wrist.shape, amusement_wrist.shape

### Downsampling

In [ ]:
baseline_wrist_sample = EDAUtils.downsample(baseline_wrist, EDAUtils.mod_samp_rate['label'], 1)
stress_wrist_sample = EDAUtils.downsample(stress_wrist, EDAUtils.mod_samp_rate['label'], 2)
amusement_wrist_sample = EDAUtils.downsample(amusement_wrist, EDAUtils.mod_samp_rate['label'], 3) # 0 ???

In [ ]:
wrist_all_samples = pd.concat([baseline_wrist_sample, stress_wrist_sample, amusement_wrist_sample])
wrist_all_samples = pd.concat([wrist_all_samples.drop('label', axis=1), pd.get_dummies(wrist_all_samples['label'])], axis=1)

### Formatting

In [ ]:
wrist_all_samples['label'] = (wrist_all_samples[3].astype(str) + wrist_all_samples[1].astype(str) + wrist_all_samples[2].astype(str)).apply(lambda x: x.index('1'))
wrist_all_samples.drop([3, 1, 2], axis=1, inplace=True)
wrist_all_samples.reset_index(drop=True, inplace=True)
wrist_all_samples.head()

In [ ]:
wrist_all_samples.shape[0], wrist_all_samples[wrist_all_samples['label'] == 1].shape[0], wrist_all_samples[wrist_all_samples['label'] == 2].shape[0], wrist_all_samples[wrist_all_samples['label'] == 3].shape[0]

In [ ]:
wrist_all_samples.columns.values